<a href="https://colab.research.google.com/github/TienLungSun/RL-Unity-ML-Agent/blob/main/7.%20PPO%2BCuriosity%20to%20learn%20Pyramid%20(2)(MLAgent_10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from mlagents_envs.environment import UnityEnvironment

In [ ]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cuda NVIDIA GeForce RTX 3060


### Global variables are in captial

In [ ]:
N_STATES  = 172  # 56+56+56+4
N_ACTIONS = 1     # 1 branch with 5 VALUES
N_AGENTS = 3

HIDDEN_UNITS = 512

BATCH_SIZE = 64  
BUFFER_SIZE = 12000
LEARNING_RATE = 0.0003
BETA = 0.001
EPSILON = 0.2
LAMBD = 0.99
N_EPOCH = 3

GAMMA = 0.99

MAX_STEPS = 5000 #50000
TIME_HORIZON = 50 #1000 

In [ ]:
ENV = BEHAVIOR_NAME = NET = None

In [ ]:
# tensor arrays to store (s,a,r,s1) data collected from N_AGENTS in TIME_HORIZON steps
a = torch.FloatTensor([[0]]*N_AGENTS ) #create NoAgents by 1 tensor
b = torch.FloatTensor([[0]*N_ACTIONS ]*N_AGENTS ) 
c = torch.FloatTensor([[0]*N_STATES  ]*N_AGENTS ) 

VALUES =REWARDS = MASKS = [a]*TIME_HORIZON
LOG_PROBS = ACTIONS = [b]*TIME_HORIZON
STATES = NEXT_STATES = [c]*TIME_HORIZON

### Actor, critic, forward prediction NN

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)

In [ ]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(N_STATES, HIDDEN_UNITS),
            nn.LayerNorm(HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, HIDDEN_UNITS),
            nn.LayerNorm(HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(N_STATES, HIDDEN_UNITS),
            nn.LayerNorm(HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, HIDDEN_UNITS),
            nn.LayerNorm(HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, N_ACTIONS)
        )
        
        self.fNET = nn.Sequential(
            nn.Linear(N_STATES, HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, HIDDEN_UNITS),
            nn.Linear(HIDDEN_UNITS, N_STATES)
        )
        
        self.log_std = nn.Parameter(torch.ones(1, N_ACTIONS) * 0.0)
        self.apply(init_weights)
    
    def forward(self, x):
        pred_next_state = self.fNET(x)
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value, pred_next_state

In [ ]:
NET = Net().to(device)

In [ ]:
optimizer = optim.Adam(NET.parameters(), lr=LEARNING_RATE )

### function to interacts with Unity NoSteps

In [ ]:
def Interact_with_Unity_one_step (DecisionSteps):
    # DecisionSteps.obs=[(NoAgents, 56), (NoAgents, 56), (NoAgents, 56), (NoAgents, 4)]
    # ENV and NET are global variables 
    s = torch.FloatTensor(DecisionSteps.obs[0])
    for i in range(1, 4):
        s = torch.cat((s, torch.FloatTensor(DecisionSteps.obs[i])), 1)
    dist, value, _ = NET(s.to(device))
    a = dist.sample() 
    log_prob = dist.log_prob(a)
    actionCFD = dist.cdf(a)
    actionIdx = GenerateActionIndex(a, actionCFD)
    ENV.set_actions(BEHAVIOR_NAME, actionIdx)   
    ENV.step()
    return s, value, a, log_prob

In [ ]:
def Collect_REWARDS_and_MASKS (step, AgentSteps, flag): 
    #flag=1:decision, 0: terminal steps
    #REWARDS, MASKS, NEXT_STATES are gloable variables
    r = AgentSteps.reward
    r = torch.FloatTensor(r).unsqueeze(1)
    s = torch.FloatTensor(AgentSteps.obs[0])
    for i in range(1, 4):
        s = torch.cat((s, torch.FloatTensor(AgentSteps.obs[i])), 1)

    s = torch.FloatTensor(s).to(device) 
    for idx in range(len(AgentSteps)):
        AgentID = AgentSteps.agent_id[idx]
        REWARDS[step][AgentID]=r[idx]
        MASKS[step][AgentID]= flag
        NEXT_STATES[step][AgentID]=s[idx]

In [ ]:
def GenerateActionIndex(action, actionCFD): 
    actionIdx = []
    for agentID in range(N_AGENTS):
        agentActionIdx = []
        for idx, value in enumerate(action[agentID]):
            cfd = float(actionCFD[agentID][idx])
            if(cfd<0.2):
                agentActionIdx.append(0)
            elif(cfd<0.4):
                agentActionIdx.append(1)
            elif(cfd<0.6):
                agentActionIdx.append(2)
            elif(cfd<0.8):
                agentActionIdx.append(3)
            else:
                agentActionIdx.append(4)
        actionIdx.append(agentActionIdx)
    return np.array(actionIdx)

Interact with Unity to collect data<br /> 
When the agent's Decision period >1, there will be cases where some agents do not have decision steps. We will collect data only when all agents have decision steps, i.e., len(DecisionSteps)==NoAgents

In [ ]:
def collect_training_data ():
    #ENV, BEHAVIOR_NAME are gloabl variables
    #states, ACTIONS, LOG_PROBS, VALUES, REWARDS, MASKS, NEXT_STATES are global variables (tensor array)
    step = 0  #index 0
    DecisionSteps, TerminalSteps = ENV.get_steps(BEHAVIOR_NAME)
    while(step < TIME_HORIZON): #try to run NoSteps good steps
        #if we have no decision agents,then continue next loop without increase step
        if(len(DecisionSteps) == 0): 
            ENV.reset() 
            DecisionSteps, TerminalSteps = ENV.get_steps(BEHAVIOR_NAME)
            continue #continue next while loop without increase step
        
        # Interacts with Unity one step
        s, value, a, log_prob = Interact_with_Unity_one_step (DecisionSteps)
        NextDecisionSteps, NextTerminalSteps = ENV.get_steps(BEHAVIOR_NAME)

        #if this or next decision step misses some agents, then do not collect data
        if(len(DecisionSteps)!= N_AGENTS or len(NextDecisionSteps)!= N_AGENTS): 
            DecisionSteps, TerminalSteps = NextDecisionSteps, NextTerminalSteps
            continue      #continue next while loop without increase step
        
        #else this and next decision steps includes all agents, collect (s, a, r, s1)
        for idx in range(len(DecisionSteps)):
            #find decision agents and record their state, value and ACTIONS
            AgentID = DecisionSteps.agent_id[idx]
            STATES[step][AgentID]=s[idx]
            VALUES[step][AgentID]=value[idx]
            ACTIONS[step][AgentID]=a[idx]
            LOG_PROBS[step][AgentID]=log_prob[idx]

        #collect reward of this action from next decision and terminal steps
        if(len(NextTerminalSteps) >0):
            #if next step has terminal agents, then collect terminal agents first
            Collect_REWARDS_and_MASKS(step, NextTerminalSteps, 0)
        else:  #else collect r and next state from decision steps
            Collect_REWARDS_and_MASKS(step, NextDecisionSteps, 1)
        
        step = step + 1 #increase step and run next while
        DecisionSteps, TerminalSteps = NextDecisionSteps, NextTerminalSteps

### GAE

In [ ]:
def compute_gae(next_value):
    value1 = VALUES + [next_value.cpu()]
    gae = 0
    returns = []
    for step in reversed(range(TIME_HORIZON )):
        delta = REWARDS[step] + GAMMA*value1[step + 1]*MASKS[step]-value1[step]
        gae = delta + GAMMA*LAMBD*MASKS[step]*gae
        returns.insert(0, gae + VALUES[step])
    return returns

### PPO optimization

In [ ]:
def ppo_iter():
    buffer_size = MERGED_STATES.size(0)
    for _ in range(buffer_size// BATCH_SIZE ):
        rand_ids = np.random.randint(0, buffer_size, BATCH_SIZE )
        yield MERGED_STATES[rand_ids, :], MERGED_ACTIONS[rand_ids, :], MERGED_NEXT_STATES[rand_ids, :],\
              MERGED_LOG_PROBS[rand_ids, :], MERGED_RETURNS[rand_ids, :], MERGED_ADVANTAGES[rand_ids, :]

In [ ]:
def ppo_curiosity_update():
    print("epoch:")
    for epoch in range(N_EPOCH):
        print(epoch, end = ", ")
        for b_s, b_a, b_s_, b_old_LOG_PROBS, b_return, b_advantage in ppo_iter():
            dist, value, pred_b_s_ = NET(b_s.to(device))            
            forward_loss = (pred_b_s_ - b_s_.to(device)).pow(2).mean()
            
            #update Actor-Critic
            critic_loss = (b_return.to(device) - value).pow(2).mean()
            entropy = dist.entropy().mean()
            b_a_new = dist.sample()
            b_new_LOG_PROBS = dist.log_prob(b_a_new)
            ratio = (b_new_LOG_PROBS - b_old_LOG_PROBS.to(device)).exp()
            surr1 = ratio * b_advantage.to(device)
            surr2 = torch.clamp(ratio, 1.0-EPSILON, 1.0+EPSILON) * b_advantage.to(device)
            actor_loss  = - torch.min(surr1, surr2).mean()
            loss = (0.5 * critic_loss + actor_loss - 0.001 * entropy) + 0.02*forward_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return float(critic_loss), float(actor_loss), float(forward_loss)

### Try one training iteration

In [ ]:
ENV = UnityEnvironment(file_name= None, base_port=5004)

In [ ]:
ENV.reset()
behavior_names = list(ENV.behavior_specs.keys())
BEHAVIOR_NAME = behavior_names[0]
print(BEHAVIOR_NAME)

Pyramids?team=0


In [ ]:
collect_training_data()
print(len(LOG_PROBS), LOG_PROBS[0].shape)
print(len(VALUES), VALUES[0].shape)
print(len(REWARDS), REWARDS[0].shape)
print(len(MASKS), MASKS[0].shape)
print(len(STATES), STATES[0].shape)
print(len(ACTIONS), ACTIONS[0].shape)
print(len(NEXT_STATES), NEXT_STATES[0].shape)

50 torch.Size([3, 1])
50 torch.Size([3, 1])
50 torch.Size([3, 1])
50 torch.Size([3, 1])
50 torch.Size([3, 172])
50 torch.Size([3, 1])
50 torch.Size([3, 172])


In [ ]:
# send last next state to calculate value
_, next_value, _ = NET(NEXT_STATES[-1].to(device)) 

In [ ]:
RETURNS = compute_gae(next_value)

In [ ]:
MERGED_RETURNS   = torch.cat(RETURNS).detach()
MERGED_LOG_PROBS = torch.cat(LOG_PROBS).detach()
MERGED_VALUES    = torch.cat(VALUES).detach()
MERGED_STATES    = torch.cat(STATES) 
MERGED_NEXT_STATES   = torch.cat(NEXT_STATES) 
MERGED_ACTIONS   = torch.cat(ACTIONS)
MERGED_ADVANTAGES = MERGED_RETURNS - MERGED_VALUES

In [ ]:
print(len(MERGED_RETURNS), MERGED_RETURNS[0].shape)
print(len(MERGED_LOG_PROBS), MERGED_LOG_PROBS[0].shape)
print(len(MERGED_VALUES), MERGED_VALUES[0].shape)
print(len(MERGED_STATES), MERGED_STATES[0].shape)
print(len(MERGED_NEXT_STATES), MERGED_NEXT_STATES[0].shape)
print(len(MERGED_ACTIONS), MERGED_ACTIONS[0].shape)
print(len(MERGED_ADVANTAGES), MERGED_ADVANTAGES[0].shape)

150 torch.Size([1])
150 torch.Size([1])
150 torch.Size([1])
150 torch.Size([172])
150 torch.Size([172])
150 torch.Size([1])
150 torch.Size([1])


In [ ]:
critic_loss, actor_loss, forward_loss = ppo_curiosity_update()
print(critic_loss, actor_loss, forward_loss)

epoch:
0, 1, 2, 137.13279724121094 -10.685394287109375 3.925424337387085


In [ ]:
ENV.close()

# Interact with Unity for N steps

In [ ]:
ENV = UnityEnvironment(file_name= None, base_port=5004)

In [ ]:
ENV.reset()
BEHAVIOR_NAME = list(ENV.behavior_specs.keys())
BEHAVIOR_NAME = BEHAVIOR_NAME[0]

In [ ]:
ActorLossLst = []
CriticLossLst = []
ForwardLossLst = []
frame_idx  = 0 

while (frame_idx < MAX_STEPS):
    print("\nframe idx = ", frame_idx)
    print("Interacts with Unity to collect training data")
    collect_training_data()
    _, next_value, _ = NET(NEXT_STATES[-1].to(device)) 
    
    print("Compute GAE of these training data set")
    RETURNS = compute_gae(next_value)
    MERGED_RETURNS   = torch.cat(RETURNS).detach()
    MERGED_LOG_PROBS = torch.cat(LOG_PROBS).detach()
    MERGED_VALUES    = torch.cat(VALUES).detach()
    MERGED_STATES    = torch.cat(STATES) 
    MERGED_NEXT_STATES    = torch.cat(NEXT_STATES) 
    MERGED_ACTIONS   = torch.cat(ACTIONS)
    MERGED_ADVANTAGES = MERGED_RETURNS - MERGED_VALUES
    
    print("Optimize NN with PPO and curiosity")
    critic_loss, actor_loss, forward_loss = ppo_curiosity_update()
    CriticLossLst.append(critic_loss)
    ActorLossLst.append(actor_loss)
    ForwardLossLst.append(forward_loss)

    frame_idx += TIME_HORIZON


frame idx =  0
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  50
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  100
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  150
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  200
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  250
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  300
Interacts with Unity to collect training data
Compute GAE of 

Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2700
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2750
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2800
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2850
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2900
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curiosity
epoch:
0, 1, 2, 
frame idx =  2950
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO and curios

In [ ]:
ENV.close()